In [1]:
import os
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.sequential import SequentialChain

llm = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key = openai_api_key,
    base_url= "https://api.chatanywhere.tech/v1"
    # organization="...",
    # other params...
)

In [6]:
def experimental_steps_chain():
    # 1. 定义第一个链：给出实验的步骤
    experimental_template = "我需要完成这个实验： {experimental}。请给出这个实验所需的步骤。"  
    experimental_prompt = PromptTemplate(input_variables=["experimental"], template=experimental_template)      
    experimental_chain = LLMChain(llm=llm, prompt=experimental_prompt, output_key="experimental_steps")  

    # 定义第二个链：基于上述实验步骤提取每个步骤中的物体和操作 
    steps_template = "根据实验步骤：{experimental_steps}，按照出现顺序提取（操作步骤，物体，操作，被操作物体）四元组"  
    steps_prompt = PromptTemplate(input_variables=["experimental_steps"], template=steps_template)  
    steps_chain = LLMChain(llm=llm, prompt=steps_prompt, output_key="steps")  

    # 串联两个链  
    sequential_chain = SequentialChain(  
        chains=[experimental_chain, steps_chain],  
        input_variables=["experimental"],  # 起始输入  
        output_variables=["steps"]  # 最终输出  
    )  
    return sequential_chain


In [7]:
test_experimental_steps_chain = experimental_steps_chain()

In [8]:
result = test_experimental_steps_chain.run("高锰酸钾制取氧气")

In [27]:
result

'1. (准备设备, 三脚架, 放置, 稳定的工作台上)\n2. (准备设备, 网眼, 放置, 三脚架上)\n3. (准备设备, 烧杯或坩埚, 放置, 网眼上)\n4. (装载高锰酸钾, 高锰酸钾晶体, 放入, 烧杯或坩埚中)\n5. (设置导管, 导管, 连接, 烧杯口)\n6. (设置导管, 导管, 连接, 水槽中的倒置烧杯或气体收集袋)\n7. (加热, 酒精灯或煤气灯, 点燃, 无)\n8. (加热, 火焰, 置于, 烧杯或坩埚下方)\n9. (加热, 高锰酸钾, 观察, 变黑并开始分解)\n10. (收集氧气, 氧气, 导入, 水槽中的倒置烧杯或气体收集袋中)\n11. (结束加热, 火源, 熄灭, 无)\n12. (实验后处理, 设备, 确保, 完全冷却后)\n13. (实验后处理, 高锰酸钾残留物, 进行, 危险废物处理)'